# Text to Cypher Retriever



Vector and full text retrievers are great for finding relevant data based on semantic similarity or keyword matching.

To answer more specific questions, you may need to perform more complex queries to find data relating to specific nodes, relationships, or properties.

For example, you want to find:

    The age of an actor.

    Who acted in a movie.

    Movie recommendations based on rating.

Text to Cypher retrievers allow you to convert natural language queries into Cypher queries that can be executed against the graph.
```
[User Query]
"What year was the movie Babe released?"

[Generated Cypher Query]
MATCH (m:Movie)
WHERE m.title = 'Babe'
RETURN m.released

[Cypher Result]
1995

[LLM Response]
"The movie Babe was released in 1995."
```

In this lesson, you will create a text to Cypher retriever that will generate context based on the user’s query and the graph schema.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import Text2CypherRetriever

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Create Cypher LLM
# The temperature is set to 0. When generating Cypher queries, you want the output to be deterministic and precise.
t2c_llm = OpenAILLM(
    model_name="gpt-4o", 
    model_params={"temperature": 0}
)


# Build the retriever
retriever = Text2CypherRetriever(
    driver=driver,
    llm=t2c_llm,
)

llm = OpenAILLM(model_name="gpt-4o")
rag = GraphRAG(retriever=retriever, llm=llm)

query_text = "Which movies did Hugo Weaving star in?"

response = rag.search(
    query_text=query_text,
    return_context=True
    )

print(response.answer)
print("CYPHER :", response.retriever_result.metadata["cypher"])
print("CONTEXT:", response.retriever_result.items)

Hugo Weaving starred in several movies, including "The Matrix" series, "The Lord of the Rings" trilogy, "V for Vendetta," "The Hobbit" trilogy, "The Adventures of Priscilla, Queen of the Desert," and "Transformers" series, among others. Unfortunately, the provided context does not list specific movies, so this list is based on general knowledge.
CYPHER : MATCH (a:Actor {name: "Hugo Weaving"})-[:ACTED_IN]->(m:Movie)
RETURN m.title
CONTEXT: [RetrieverResultItem(content="<Record m.title='Babe'>", metadata=None)]


# Example Queries

To improve the accuracy of the generated Cypher queries, you can provide examples of queries and an appropriate Cypher query.

If you wanted to find data related to movie ratings, you will could provide an example query that demonstrates how to retrieve ratings from the graph:

```cypher
USER INPUT: 'Get user ratings for a movie?'

QUERY: MATCH ()-[r:RATED]->(m:Movie)
       WHERE m.title = 'Movie Title'
       RETURN r.rating

```
Create a list of the example queries:

In [2]:
# Cypher examples as input/query pairs
examples = [
    "USER INPUT: 'Get user ratings for a movie?' QUERY: MATCH (u:User)-[r:RATED]->(m:Movie) WHERE m.title = 'Movie Title' RETURN r.rating"
]

In [3]:
# Build the retriever
retriever = Text2CypherRetriever(
    driver=driver,
    llm=t2c_llm,
    examples=examples,
)

In [4]:
response = rag.search(
    query_text=query_text,
    return_context=True
    )

print(response.answer)
print("CYPHER :", response.retriever_result.metadata["cypher"])
print("CONTEXT:", response.retriever_result.items)

Hugo Weaving starred in several movies, including "The Matrix" series, "The Lord of the Rings" trilogy, "V for Vendetta," "The Hobbit" trilogy, and "Captain America: The First Avenger."
CYPHER : MATCH (a:Actor {name: "Hugo Weaving"})-[:ACTED_IN]->(m:Movie)
RETURN m.title
CONTEXT: [RetrieverResultItem(content="<Record m.title='Babe'>", metadata=None)]


## Schema

The `TextToCypherRetriever` will automatically read the whole graph schema from the database.

You can provide a custom schema to the retriever if you want to limit the nodes, relationships and properties that are used to generate Cypher queries. Limiting the scope of the schema can help improve the accuracy of the generated Cypher queries, particularly if the graph contains a lot of nodes and relationships.

Create a textual representation of the graph schema:

In [5]:
# Specify your own Neo4j schema
neo4j_schema = """
Node properties:
Person {name: STRING, born: INTEGER}
Movie {tagline: STRING, title: STRING, released: INTEGER}
Genre {name: STRING}
User {name: STRING}

Relationship properties:
ACTED_IN {role: STRING}
RATED {rating: INTEGER}

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:User)-[:RATED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
"""

In [6]:
# Build the retriever
retriever = Text2CypherRetriever(
    driver=driver,
    llm=t2c_llm,
    neo4j_schema=neo4j_schema,
    examples=examples,
)

In [7]:
response = rag.search(
    query_text=query_text,
    return_context=True
    )

print(response.answer)
print("CYPHER :", response.retriever_result.metadata["cypher"])
print("CONTEXT:", response.retriever_result.items)

Hugo Weaving has starred in several movies, including:

1. "The Matrix" series, where he played Agent Smith.
2. "The Lord of the Rings" series, portraying Elrond.
3. "V for Vendetta" as V.
4. "Captain America: The First Avenger" as Red Skull.
5. "The Hobbit" series, again as Elrond.
6. "Cloud Atlas" in multiple roles.
7. "Happy Feet" as the voice of Noah the Elder.
8. "Transformers" as the voice of Megatron.

These are just a few examples of Hugo Weaving's filmography.
CYPHER : MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE a.name = 'Hugo Weaving'
RETURN m.title
CONTEXT: [RetrieverResultItem(content="<Record m.title='Babe'>", metadata=None)]


In [8]:
driver.close()

# GenAI Frameworks



A variety of open-source and community-supported frameworks are available to help you integrate Neo4j with generative AI and large language models (LLMs).

These frameworks support use cases such as:

- Retrieval-augmented generation (RAG).

- Agentic workflows.

- Knowledge graph construction.

Typically these frameworks include:

- LLM usage, prompt and output management.

- Embedding model integration .

- Vector and database integration (including Neo4j).

- RAG (Retrieval-Augmented Generation) workflows.

- Agentic workflows and orchestration.

- Monitoring, observability, and deployment tools.

## Popular GenAI Frameworks for Neo4j

- [LangChain (Python)](https://www.langchain.com/) - A leading open-source framework for building LLM-powered applications, with strong support for Neo4j as a vector store and knowledge graph.

- [LangChainJS - The JavaScript/TypeScript](https://www.langchain.com/) version of LangChain, enabling GenAI workflows in Node.js and browser environments.

- [LlamaIndex](https://www.llamaindex.ai/) - A data framework for connecting LLMs to external data, with connectors for Neo4j to support RAG and knowledge graph use cases.

- [Spring AI](https://spring.io/projects/spring-ai) - A Spring ecosystem project for integrating AI capabilities into Java applications, including Neo4j support.

- [Langchain4j](https://docs.langchain4j.dev/) - A Java version of LangChain, supporting Neo4j integration for LLM and RAG workflows.

- [Haystack](https://haystack.deepset.ai/) - An open-source framework for building search and question-answering systems, with Neo4j integration for graph-based retrieval.

- [Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/overview/) - A Microsoft open-source orchestration library for AI workflows, supporting Neo4j as a data source.

- [DSPy](https://dspy.ai/) - A framework for programming and optimizing LLM pipelines, with Neo4j connectors.

You can learn more in the [Neo4j GenAI Frameworks documentation](https://neo4j.com/labs/genai-ecosystem/genai-frameworks/?_gl=1*1x84758*_gcl_au*MjEzNTI4NjkxNy4xNzU3MjU4NDMzLjc4MDQ1OTczLjE3NTg0MTY3NjUuMTc1ODQxNjc2NA..*_ga*MTkzMzgxNTk1LjE3NTcyNTg0MzQ.*_ga_DL38Q8KGQC*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjg2OTQ4JGo1OSRsMCRoMA..*_ga_DZP8Z65KK4*czE3NjMyNjQ0MzMkbzQ3JGcxJHQxNzYzMjg2OTQ4JGo1OSRsMCRoMA..).

Frameworks support your GenAI application development and help you build complex workflows that integrate Neo4j with LLMs and other AI components.